In [30]:
import numpy as np
from seemps.hamiltonians import ConstantNNHamiltonian
from seemps.optimization import dmrg
from seemps.state import MPS, product_state
from seemps.tools import σx, σy, σz

σp = 0.5 * (σx + 1j*σy)
σm = 0.5 * (σx - 1j*σy)


In [23]:
L = 10  # Number of unit cells
N_sites = 2 * L

# Parameters (example)
omega_c = 4.0
omega_q = 1.0
g_Φ = 0.5
g_c = 1.0
g_q = 1.0
g_qc = np.sqrt(g_q * g_c)


In [24]:
H = ConstantNNHamiltonian(size=N_sites, dimension=2)


In [25]:
for n in range(L):
    ia = 2*n   # resonator
    ib = 2*n+1 # qubit

    H.add_local_term(ia, (omega_c/2)*σz)
    H.add_local_term(ib, (omega_q/2)*σz)

    # Rung coupling g_Φ
    H.add_interaction_term(ia, g_Φ * σp, σm)  # resonator to qubit rung coupling
    H.add_interaction_term(ia, g_Φ * σm, σp)  # Hermitian conjugate


In [26]:
for n in range(L-1):
    ia, ib = 2*n, 2*n+1
    ia_next, ib_next = ia+2, ib+2  # next unit cell

    # resonator-resonator hopping (g_c)
    H.add_interaction_term(ia, g_c * σp, σm)
    H.add_interaction_term(ia, g_c * σm, σp)

    # qubit-qubit hopping (-g_q)
    H.add_interaction_term(ib, -g_q * σp, σm)
    H.add_interaction_term(ib, -g_q * σm, σp)

    # cross-coupling resonator-qubit (sqrt(g_q*g_c))
    H.add_interaction_term(ia, g_qc * σp, σm)  # a_n → b_{n+1}
    H.add_interaction_term(ia, g_qc * σm, σp)  

    H.add_interaction_term(ib, -g_qc * σp, σm) # b_n → a_{n+1}
    H.add_interaction_term(ib, -g_qc * σm, σp)


In [27]:
H_matrix = H.to_matrix()
print(H_matrix.shape)


(1048576, 1048576)


In [28]:
H_mpo = H.to_mpo()
print(H_mpo)


In [29]:
result = dmrg(H, guess=product_state(np.ones(2) / np.sqrt(2), N_sites))
print(f"DMRG energy = {result.energy}")


DMRG energy = -34.15725125878252


In [34]:
from seemps.operators import MPO

# build the N‐site identity MPO
d = 2
identity_tensors = [ np.eye(d).reshape(1, d, d, 1)
                     for _ in range(N_sites) ]
identity_mpo = MPO(identity_tensors)


In [35]:

# Shift your Hamiltonian explicitly
E_target = 0.0  # or your desired target energy
H_shifted = H_mpo - E_target * identity_mpo

In [36]:

# Square it explicitly
H_shifted_squared = H_shifted @ H_shifted


TypeError: Cannot multiply MPO with <seemps.operators.mposum.MPOSum object at 0x000001F15983EC10>

In [31]:
# Target energy estimate ( band-center)
E_target = 0.0

# Shifted Hamiltonian: H - E_target * I
H_shifted = H_mpo - E_target * H_mpo.identity()

# Squared Hamiltonian to create (H - E_target)^2 MPO
H_shifted_squared = H_shifted @ H_shifted


AttributeError: 'MPO' object has no attribute 'identity'

In [ ]:
# Initial random MPS
psi0 = MPS.random([2]*N_sites, max_bond_dimension=20)

# Find ground state of the shifted squared Hamiltonian
energy_shifted, psi_midgap = ground_state(H_shifted_squared, psi0, max_bond_dimension=50, tol=1e-6)

print("Ground state energy of shifted squared H:", energy_shifted)
